In [21]:
#web scraping for data
#to build database of windows for training
import requests, lxml, re, json, urllib
from bs4 import BeautifulSoup

headers = {
    "User-Agent":
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
}

params = {
    "q": "window hdb",
    "tbm": "isch",
    "ijn": "0",
}

html = requests.get("https://www.google.com/search", params=params, headers=headers)
soup = BeautifulSoup(html.text, 'lxml')


def get_images_data():

    all_script_tags = soup.select('script')

    # # https://regex101.com/r/48UZhY/4
    matched_images_data = ''.join(re.findall(r"AF_initDataCallback\(([^<]+)\);", str(all_script_tags)))

    matched_images_data_fix = json.dumps(matched_images_data)
    matched_images_data_json = json.loads(matched_images_data_fix)

    # https://regex101.com/r/pdZOnW/3
    matched_google_image_data = re.findall(r'\[\"GRID_STATE0\",null,\[\[1,\[0,\".*?\",(.*),\"All\",', matched_images_data_json)

    # https://regex101.com/r/NnRg27/1
    matched_google_images_thumbnails = ', '.join(
        re.findall(r'\[\"(https\:\/\/encrypted-tbn0\.gstatic\.com\/images\?.*?)\",\d+,\d+\]',
                   str(matched_google_image_data))).split(', ')

    print('Google Image Thumbnails:')  # in order
    for fixed_google_image_thumbnail in matched_google_images_thumbnails:
        # https://stackoverflow.com/a/4004439/15164646 comment by Frédéric Hamidi
        google_image_thumbnail_not_fixed = bytes(fixed_google_image_thumbnail, 'ascii').decode('unicode-escape')

        # after first decoding, Unicode characters are still present. After the second iteration, they were decoded.
        google_image_thumbnail = bytes(google_image_thumbnail_not_fixed, 'ascii').decode('unicode-escape')
        print(google_image_thumbnail)
        

    # removing previously matched thumbnails for easier full resolution image matches.
    removed_matched_google_images_thumbnails = re.sub(
        r'\[\"(https\:\/\/encrypted-tbn0\.gstatic\.com\/images\?.*?)\",\d+,\d+\]', '', str(matched_google_image_data))

    # https://regex101.com/r/fXjfb1/4
    # https://stackoverflow.com/a/19821774/15164646
    matched_google_full_resolution_images = re.findall(r"(?:'|,),\[\"(https:|http.*?)\",\d+,\d+\]",
                                                       removed_matched_google_images_thumbnails)


    print('\nFull Resolution Images:')  # in order
    count = 34
    for index, fixed_full_res_image in enumerate(matched_google_full_resolution_images):
        original_size_img_not_fixed = bytes(fixed_full_res_image, 'ascii').decode('unicode-escape')
        original_size_img = bytes(original_size_img_not_fixed, 'ascii').decode('unicode-escape')
        print(original_size_img)

        # ------------------------------------------------
        # Download original images

        # print(f'Downloading {index} image...')

        opener=urllib.request.build_opener()
        opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582')]
        urllib.request.install_opener(opener)

        urllib.request.urlretrieve(original_size_img, f'./ruler-img/ruler-{count}.jpg')
        count += 1



#get_images_data()

In [ ]:
#load manual labelled data for windows
from roboflow import Roboflow
rf = Roboflow(api_key="XIkFjSCMqTJ3giSB84K3")
project = rf.workspace().project("window-detection")
dataset = project.version(1).download("yolov5")

In [ ]:
#train command for model
!python3 ./yolov5/train.py --data ./Window-Detection-1/data.yaml --weights yolov5s.pt --img 640

In [34]:
#run on image
#source accepts folder of images or an image
!python3 ./yolov5/detect.py --weights ./yolov5/runs/train/exp2/weights/best.pt \
                            --img 640 --conf 0.5 --source ./actual-data/ruler-2.jpeg --save-txt

#save-txt generates a text file of class, x, y, width, height in each row for each object
#assuming image is plotted into 1x1 graph 

detect: weights=['./yolov5/runs/train/exp2/weights/best.pt'], source=./actual-data/ruler-2.jpeg, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-127-g554f782 torch 1.10.0 CPU

Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /Users/yanch/Documents/personal/uni/fintech/project-2/actual-data/ruler-2.jpeg: 640x384 1 ruler, Done. (0.208s)
Speed: 2.6ms pre-process, 207.7ms inference, 18.4ms NMS per image at shape (1, 3, 640, 640)
Results saved to yolov5/runs/detect/exp16
1 labels saved to yolov5/runs/detect/exp16/labels


In [22]:
#train dataset to identify ruler
params = {
    "q": "ruler",
    "tbm": "isch",
    "ijn": "0",
}

html = requests.get("https://www.google.com/search", params=params, headers=headers)
soup = BeautifulSoup(html.text, 'lxml')
get_images_data()

Google Image Thumbnails:
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTKlw7rSt2JRxyIJmajZx81k89v9Tx9tiJnmW1lWrgYf52iXddRqrLUmCYuJ6t6JpXVnOA&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRIU7IQ2qIl54jDAjqIf_DrgxEatM5n5VDgleySE75YZ_-Hegib08brHQ-gDd82kpbhuQ8&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS08hoW8zQe9OvfnnJwDEYJs13xjgO2in1dCo_I3Htrs7bK6uTejX1RZT-bsfu837wQ0A&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTASBFdhTQEY8B8K6VwpENWYasc3Zeg-N760fmtIFtqVBtSwCMLx10tStI6u6WDZnDlJd4&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRBsAqGAjTZkKigDuvht4OTdJM-tgcbBvYsjWboz1FlKNcIQGOIGEVow7Hl2MNd7b2XZKE&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR_aQC7dPrhyEClMv6mESKDIE51UVy81uVQXWO60qXfzoiRI8xYy7o85Ca0kTNC7w-BZ14&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSvSftEd_3gn8eeHe9ZFdjGpUe318txvPD1BM9a9Z0xpRXyWbFw2exoYP4E4CyZHUPmv_A&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=t

https://www.ginifab.com/feeds/cm_to_inch/img/ruler.png
https://m.media-amazon.com/images/I/71tWGvQ+xCL._AC_SY355_.jpg
https://media.istockphoto.com/photos/20cm-wooden-ruler-on-a-white-background-picture-id186348471
https://m.media-amazon.com/images/I/51QzEF5bi-L._AC_SY355_.jpg
https://m.media-amazon.com/images/I/71Yz+YiQG3L._AC_SY355_.jpg
https://image.shutterstock.com/image-vector/ruler-inches-cm-scale-on-260nw-1180940488.jpg
https://res.cloudinary.com/rsc/image/upload/b_rgb:FFFFFF,c_pad,dpr_1.0,f_auto,h_337,q_auto,w_600/c_pad,h_337,w_600/F1779573-03?pgw=1
https://imgprd19.hobbylobby.com/sys-master/migrated/he0/h1f/h00/9326420525086/144519-1116[9].jpg
https://media.officedepot.com/images/f_auto,q_auto,e_sharpen,h_450/products/724522/724522_o03_rlr_6_in_blu_100714/724522_o03_rlr_6_in_blu_100714
https://target.scene7.com/is/image/Target/GUEST_4a683c0e-dad3-4848-b2fd-64e148dde3e5?wid=488&hei=488&fmt=pjpeg
https://image.shutterstock.com/image-photo/lifetime-12-inch-ruler-260nw-545801860.j

HTTPError: HTTP Error 500: Internal Server Error

In [24]:
from roboflow import Roboflow
rf = Roboflow(api_key="XIkFjSCMqTJ3giSB84K3")
project = rf.workspace().project("window-detection")
dataset = project.version(2).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Window-Detection-2 in yolov5pytorch:: 100%|█| 


In [25]:
#train command for new model to include rulers
!python3 ./yolov5/train.py --data ./Window-Detection-2/data.yaml --weights ./yolov5/runs/train/exp/weights/best.pt --img 640

train: weights=./yolov5/runs/train/exp/weights/best.pt, cfg=, data=./Window-Detection-2/data.yaml, hyp=yolov5/data/hyps/hyp.scratch.yaml, epochs=300, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (65/65), done.
remote: Total 106 (delta 65), reused 65 (delta 65), pack-reused 41
Receiving objects: 100% (106/106), 68.33 KiB | 282.00 KiB/s, done.
Resolving deltas: 100% (67/67), completed with 20 local objects.
From https://github.com/ultralytics/yolov5
   554f782..628817d  master     -> origin/master
  

     1/299        0G    0.1105   0.02831   0.02937        36       640:  20%|██ /usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
     1/299        0G     0.109    0.0283   0.03106        34       640:  40%|███/usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
     1/299        0G    0.1066   0.02769   0.03299        33       640:  60%|███/usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
     1/299   

     5/299        0G    0.0633   0.02347   0.02015        33       640:  40%|███/usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
     5/299        0G   0.07266   0.02369   0.02284        36       640:  60%|███/usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
     5/299        0G   0.07017   0.02377    0.0218        34       640:  80%|███/usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
     5/299   

     9/299        0G   0.06865   0.02274   0.02071        26       640:  60%|███/usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
     9/299        0G     0.063   0.02328   0.02009        33       640:  80%|███/usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
     9/299        0G   0.06222   0.02345   0.01956         8       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         19         20     0.0168     0.0379    0.00706    0.00175

     Epoch   gpu_mem       box       obj       cls    labels  img_size
  0%|          | 0/5 

    13/299        0G   0.05118   0.02305    0.0224        34       640:  80%|███/usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
    13/299        0G    0.0546    0.0247   0.02344        10       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         19         20     0.0317      0.312      0.022    0.00657

     Epoch   gpu_mem       box       obj       cls    labels  img_size
  0%|          | 0/5 [00:00<?, ?it/s]                                           /usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
    14/299        0G 

...

   280/299        0G   0.02455   0.00793 0.0006648        35       640:  40%|███/usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
   280/299        0G   0.02487  0.008144 0.0006593        37       640:  60%|███/usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
   280/299        0G   0.02373   0.00816 0.0007708        34       640:  80%|███/usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
   280/299   

   284/299        0G   0.01846  0.007458 0.0006131        33       640:  60%|███/usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
   284/299        0G   0.01891  0.007643 0.0005663        30       640:  80%|███/usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
   284/299        0G   0.01847  0.009202 0.0005544         7       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         19         20      0.826      0.646      0.744      0.526

     Epoch   gpu_mem       box       obj       cls    labels  img_size
  0%|          | 0/5 

   288/299        0G   0.02013  0.007744 0.0007427        38       640:  80%|███/usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
   288/299        0G   0.01764  0.007151 0.0006699         4       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         19         20      0.888      0.646      0.711      0.543

     Epoch   gpu_mem       box       obj       cls    labels  img_size
  0%|          | 0/5 [00:00<?, ?it/s]                                           /usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
   289/299        0G 

   292/299        0G   0.01604  0.007361 0.0007281         8       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         19         20      0.894      0.646      0.708      0.537

     Epoch   gpu_mem       box       obj       cls    labels  img_size
  0%|          | 0/5 [00:00<?, ?it/s]                                           /usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
   293/299        0G   0.01324  0.006757  0.000238        26       640:  20%|██ /usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
   293/299        0G 

  0%|          | 0/5 [00:00<?, ?it/s]                                           /usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
   297/299        0G   0.02194   0.01256 0.0006212        46       640:  20%|██ /usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
   297/299        0G   0.01649  0.009113 0.0004684        26       640:  40%|███/usr/local/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
   297/299   

In [ ]:
#width and height (pixels) data is available in .txt

ppm = None #pixels per metric
rule_actual = 0.15 #in metres, assuming an average 15cm ruler --> this can be set to a variable
objects = []

with open('./yolov5/runs/detect/exp-<run no.>/labels/<filename>.txt') as f:
    for line in f:
        objects.append(line.strip('\n').split())
    #extract h of rule
    #extract w and h of  at index 4 and 5 respectively
    #find ppm = ruler_h / rule_actual
    #calculate w and h of window
    

In [15]:
#alternative
#with specified distance from object and assumed camera parameters, we can find the object's real height

#formula: (object pixel height * pixel size * distance)/focal length

#26mm focal length
#assuming distance from window is 3m

fl = 0.026 #in m
d = 3
psize = 0.000065
w = 0.403125 * 640
h = 0.359167 * 640
rw = w * psize * d / fl
rh = h * psize * d / fl
print(rw, rh)

#to use real photo to calibrate psize

1.935 1.7240016000000002
